<a href="https://colab.research.google.com/github/hany606/PAI_Fall21IU/blob/main/Assignments/Assignment2/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy

In [193]:
# Import libraries
import nltk
import spacy
from spacy import displacy
import en_core_web_sm
from nltk import Tree


"""
    - [ ] deftemplate   : 
    
    ```
    Cat template has properties of color, age, and name.
    (deftemplate cat
        (slot color) (slot age) (slot name))
    ```
    
    - [ ] defrule:
    
    ```
    If there exists cat named Bob then there exists a cat named Tom.
    (defrule rule1
        (cat (name “Bob”)) => (assert (cat (name “Tom”))))
    ```

    - [ ] assert:
    ```
    There exists a cat with the name Bob.
    (assert (cat (name “Bob”)))
    ```
"""

help = "Type \n- 'exit' to go out from the program\n- 'help' to print the help menu\n- Enter any other command to translate it to CLIPS\n\
Notes:\n\t- For assertion commands use 'with' keyword for the attribute\n\t\te.g.: Cat with name bob, name here is the attribute and bob is the value\n\
\t- For assertion you should use verb 'be' in the sentence\n\
\t- For rule it is constrained with only one attribute to describe the object\n\
-------------------------------------------------------------"



class NLP:
  def __init__(self, nlp_loader=en_core_web_sm):
    self._nlp = nlp_loader.load()
    self.command_keywords = {"template": ["template", "struct", "definition", "property", "define"],
                             "rule": ["if"],
                             "assert": ["assert", "exist"]}

    self.command_root_keywords = {"template": ["have"],
                                  "rule": ["if"],
                                  "assert": ["be"]}

  def nlp(self, inp):
    return self._nlp(inp)

  def teach_me(self):
    print("?? Teach me ??")
    print("\tTeaching me a new keyword for a specific command of CLIPS from the list (template, rule, assert)\n")

    new_key = input(f"?? Enter the new key that you used for the previous commnand\n\tMost probably one from {self.all_dict['subjects']}\n?? Enter: ")
    while True:
      command = input("?? Now input the command: ").lower()
      if(command in self.command_keywords.keys()):
        self.command_keywords[command].append(new_key)
        break
      elif(command == "exit"):
        print("Did not learn new thing :(")
        return
      else:
        print("Enter the command from the list (template, rule, assert)")
    print(f"Now, the command {command} has a new keyword: {new_key}")


  def search(self, root):
    # Check the root
    for command_key in self.command_root_keywords.keys():
      for command in self.command_root_keywords[command_key]:
        if(command == root.lemma_): # if we have found a similar
          if(command_key == "template"):
            return self.clips_template(command)

          elif(command_key == "rule"):
            return self.clips_rule()

          elif(command_key == "assert"):
            return self.clips_assert()

    # Search accross the registered keywords for the commands
    for command_key in self.command_keywords.keys():
      for command in self.command_keywords[command_key]:
        if(command in self.all_dict["lemma_dict"].keys()): # if we have found a similar
          if(command_key == "template"):
            return self.clips_template(command)

          elif(command_key == "rule"):
            return self.clips_rule()

          elif(command_key == "assert"):
            return self.clips_assert()

    return None

  def _get_attributes(self):
    # For now only get all the pos_: [pobjs, conj, dobj]
    attributes = []
    for token_text in self.all_dict["tokens_dict"].keys():
      # print(token_text, self.all_dict["tokens_dict"][token_text].dep_ in ["pobjs", "conj", "dobj"], self.all_dict["tokens_dict"][token_text].pos_ in ["NOUN"])
      if(self.all_dict["tokens_dict"][token_text].dep_ in ["pobjs", "conj", "dobj"] and\
         self.all_dict["tokens_dict"][token_text].pos_ in ["NOUN"]):
        attributes.append(token_text)

    # attributes = ["name"]
    return attributes

  def _get_template_name(self, command):
    # token.head.text
    template_name = self.all_dict["tokens_dict"][list(self.all_dict["tokens_dict"].keys())[0]]  # Default
    # Iterate all over the tokens to find the template name that is attached to the keyword for template command and at the same time is noun
    # Return the last word that comforms with these conditions
    for token_text in self.all_dict["tokens_dict"].keys():
      # print(token_text,\
      #       self.all_dict["tokens_dict"][token_text].head.text in self.all_dict["lemma_dict"][command],\
      #       self.all_dict["tokens_dict"][token_text].pos_ =="NOUN",\
      #       self.all_dict["tokens_dict"][token_text].dep_ in ["compound", "ROOT"])
      if(\
        #  self.all_dict["tokens_dict"][token_text].head.text in self.all_dict["lemma_dict"][command] and\
         self.all_dict["tokens_dict"][token_text].pos_ == "NOUN" and\
         self.all_dict["tokens_dict"][token_text].dep_ in ["compound", "ROOT", "nsubj"] or\
         (self.all_dict["tokens_dict"][token_text].pos_ == "PROPN" and\
         self.all_dict["tokens_dict"][token_text].dep_ in ["compound"])):
        template_name = token_text
        break    
    return template_name

  def clips_template(self, command):
    # print("Template CLIPS TODO")
    # print(self.all_dict)
    template_name = self._get_template_name(command)
    attributes = self._get_attributes()
    # print(template_name, attributes)
    clips_attributes = [f"(slot {a})" for a in attributes]
    clips = f"(deftemplate {template_name}\n\t{' '.join(clips_attributes)})"
    # print(clips)
    return clips

  # ---------------------------------------------------------------------------------------------
  def _get_rule_objs(self):
    obj1 = None
    key_vals1 = {}
    obj2 = None
    key_vals2 = {}
    for token in self.all_dict["doc"]:  # iterate on the doc -> to get them in the same order of the sentence
      if(token.dep_ in ["dobj", "attr"] and token.pos_ in ["NOUN"] and self.all_dict["tokens_dict"][token.head.text].lemma_ in ["exist"]):
        if(obj1 is None):
          obj1 = token.text
          continue
        else:
          obj2 = token.text
          continue

      if(obj1 is not None):
        if(obj2 is None): # search for the keys & values for the obj1
          if(token.dep_ in ["acl", "amod"] and token.head.text == obj1):  # cat named bob
            print(token.head.text)
            value = None
            for token_ in self.all_dict["doc"]:  # Can be optimized instead of O(n^2)
              if(token_.head.text == token.text):
                value = token_.text
                break

            key_vals1[token.lemma_] = value

          elif(token.dep_ in ["pobj"]and\
             token.head.text in ["with"]):
            value = None
            for token_ in self.all_dict["doc"]:  # Can be optimized instead of O(n^2)
              if(token_.text == token.text):
                value = token_.text
              elif(value is not None):  # to capture the pattern "with name Bob"
                value = token_.text
                break
            key_vals1[token.lemma_] = value

        else: # search for the keys & values for the obj2
          if(token.dep_ in ["acl", "amod"] and token.head.text == obj2):  # cat named bob
            value = None
            for token_ in self.all_dict["doc"]:  # Can be optimized instead of O(n^2)
              if(token_.head.text == token.text):
                value = token_.text
            key_vals2[token.lemma_] = value
            
          elif(token.dep_ in ["pobj"]and\
             token.head.text in ["with"]):
            value = None
            print(token.lemma_, value)
            for token_ in self.all_dict["doc"]:  # Can be optimized instead of O(n^2)
              if(token_.head.text == token.text):
                value = token_.text
              elif(value is not None):  # to capture the pattern "with name Bob"
                value = token_.text
                break
            key_vals2[token.lemma_] = value
      

    return obj1, key_vals1, obj2, key_vals2

  def clips_rule(self):
    obj1, key_vals1, obj2, key_vals2 = self._get_rule_objs()
    key_vals1_compound = [f'({k} "{v}")'for k,v in key_vals1.items()]
    key_vals2_compound = [f'({k} "{v}")'for k,v in key_vals2.items()]

    # print(obj1, key_vals1, obj2, key_vals2)
    #     (defrule rule1
    #     (cat (name “Bob”)) => (assert (cat (name “Tom”))))
    clips = f"(defrule rule1 \n({obj1} {' '.join(key_vals1_compound)}) => (assert ({obj2} {' '.join(key_vals2_compound)})))"
    return clips

  # ---------------------------------------------------------------------------------------------
  def _get_asserted_obj(self):
    # nsubj, attr
    for token_text in self.all_dict["tokens_dict"].keys():
      if(self.all_dict["tokens_dict"][token_text].dep_ in ["nsubj", "attr"]and\
         self.all_dict["tokens_dict"][token_text].head.text in self.all_dict["lemma_dict"]["be"]):
        obj = token_text

    return obj

  def _get_assertion_keys_values(self):
    # keys are pobj and head.text="with"
    assertions_dict = {}
    for token_text in self.all_dict["tokens_dict"].keys():
      if(self.all_dict["tokens_dict"][token_text].dep_ in ["pobj"]and\
         self.all_dict["tokens_dict"][token_text].head.text in ["with"]):
        value = None
        for token in self.all_dict["doc"]:
          if(token.head.text == token_text):
            value = token.text

        assertions_dict[token_text] = value
    return assertions_dict

  def clips_assert(self):
    asserted_obj = self._get_asserted_obj()
    key_vals = self._get_assertion_keys_values()
    # print(asserted_obj, key_vals)
    clips_attributes = [f'({k} "{v}")' for k,v in key_vals.items()]
    clips = f"(assert {asserted_obj} {' '.join(clips_attributes)})"
    return clips
  # ---------------------------------------------------------------------------------------------

  # Source: https://stackoverflow.com/questions/36610179/how-to-get-the-dependency-tree-with-spacy
  def to_nltk_tree(self, node):
      if node.n_lefts + node.n_rights > 0:
          return Tree(node.orth_, [self.to_nltk_tree(child) for child in node.children])
      else:
          return node.orth_


  def process(self, user_input, render=False):
      # dictionaries: keys are the text and the value is the token itself
      # subjects = {}
      # actions = {}
      # objects = {}
      # tokens_dict = {}
      # lemma_dict = {} # key is the lemma and the value is the text
      self.all_dict = {"subjects": {}, # dictionaries: keys are the text and the value is the token itself
                        "actions": {}, 
                        "objects": {}, 
                        "tokens_dict": {},
                        "lemma_dict": {}, # Here: key is the lemma and the value is the text
                        "doc":None
                        }


      doc = self.nlp(user_input)
      self.all_dict["doc"] = doc
      # Store the subjs, acts, objs, and all the tokens in dictionaries
      for token in doc:
        token_text = token.text.lower()
        role = token.dep_
        if(role == "nsubj"):
          self.all_dict["subjects"][token_text] = token
        elif(role == "ROOT"):
          self.all_dict["actions"][token_text] = token
        elif(role == "dobj"):
          self.all_dict["objects"][token_text] = token

        self.all_dict["tokens_dict"][token_text] = token
        if(token.lemma_ in self.all_dict["lemma_dict"]):
          self.all_dict["lemma_dict"][token.lemma_].append(token.text)
        else:
          self.all_dict["lemma_dict"][token.lemma_] = [token.text]
        

      # ---------------------------------------------------------------------------------------------
      # Visualization
      if(render):
        for token in doc:
            print("Word = {}, Lemma = {}, PoS/Tag = {}/{}, Role = {} to [{}]".format(
                    token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.head.text))
        print("Display")
        displacy.render(doc, jupyter=True, style='dep')
        self.to_nltk_tree(list(doc.sents)[0].root).pretty_print()

          # [self.to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]
      # ---------------------------------------------------------------------------------------------



      # Search for the keywords for the commands
      search_result = self.search(list(doc.sents)[0].root)
      if(search_result is not None):
        return search_result
      self.teach_me()
      return -1

nlp = NLP()

print("--------------- Test ----------------")
print(help)

template_test = ["The cat has color, age, name, and eyes.",
                 "Cat has color, age, name, and eyes.",
                 "The cat template has properties of color, age, name, and eyes.",
                 "Cat template has properties of color, age, name, and eyes.",
                 "The cat defined by color, age, name, and eyes."]
res = [(nlp.process(temp, render=False), temp) for temp in template_test]
for r in res:
  print(r[1])
  print(r[0])
  print("-----------")

rule_test = ["If there exists a dog named Bob, then there exists a cat aged 12."]

res = [(nlp.process(rule, render=False), rule) for rule in rule_test]
for r in res:
  print(r[1])
  print(r[0])
  print("-----------")


assert_test = ["There is a dog with the name Bob, and with age 19",
               "There is a cat with the name Bob",
               "Cats are with name Bob",
               "Cat is with leg injured"]

res = [(nlp.process(assert_, render=False), assert_) for assert_ in assert_test]
for r in res:
  print(r[1])
  print(r[0])
  print("-----------")

Type 
- 'exit' to go out from the program
- 'help' to print the help menu
- Enter any other command to translate it to CLIPS
Notes:
	- For assertion commands use 'with' keyword for the attribute
		e.g.: Cat with name bob, name here is the attribute and bob is the value
	- For assertion you should use verb 'be' in the sentence
	- For rule it is constrained with only one attribute to describe the object
-------------------------------------------------------------
The cat has color, age, name, and eyes.
(deftemplate cat
	(slot color) (slot age) (slot name) (slot eyes))
-----------
Cat has color, age, name, and eyes.
(deftemplate Cat
	(slot color) (slot age) (slot name) (slot eyes))
-----------
The cat template has properties of color, age, name, and eyes.
(deftemplate cat
	(slot properties) (slot age) (slot name) (slot eyes))
-----------
Cat template has properties of color, age, name, and eyes.
(deftemplate cat
	(slot properties) (slot age) (slot name) (slot eyes))
-----------
The cat d

In [194]:
# Create UI
print(help)
nlp = NLP()

while True:
    user_input = input("Input English (1 sentence per time)\n>> ")
    # user_input = user_input.lower()

    if(user_input.lower() == "exit"):
        print("Good Bye!")
        exit()
    elif(user_input.lower() == "help"):
        print(help)
    
    clips_command = nlp.process(user_input, render=True)
    print(f"CLIPS\n>> {clips_command}")


Type 
- 'exit' to go out from the program
- 'help' to print the help menu
- Enter any other command to translate it to CLIPS
Notes:
	- For assertion commands use 'with' keyword for the attribute
		e.g.: Cat with name bob, name here is the attribute and bob is the value
	- For assertion you should use verb 'be' in the sentence
	- For rule it is constrained with only one attribute to describe the object
-------------------------------------------------------------
Input English (1 sentence per time)
>> Cat is with right hand
Word = Cat, Lemma = Cat, PoS/Tag = PROPN/NNP, Role = nsubj to [is]
Word = is, Lemma = be, PoS/Tag = AUX/VBZ, Role = ROOT to [is]
Word = with, Lemma = with, PoS/Tag = ADP/IN, Role = prep to [is]
Word = right, Lemma = right, PoS/Tag = ADJ/JJ, Role = amod to [hand]
Word = hand, Lemma = hand, PoS/Tag = NOUN/NN, Role = pobj to [with]
Display


     is      
  ___|____    
 |       with
 |        |   
 |       hand
 |        |   
Cat     right

CLIPS
>> (assert cat (hand "right"))


KeyboardInterrupt: ignored